Functions for generating landscape, agent, and individual agent scores:

Written by Udi Aharoni  
Based on "Diversity, Ability, and Expertise in Epistemic Communities"
Patrick Grim, Daniel J. Singer, Aaron Bramson, Bennett Holman, Sean McGeehan, and William J. Berger

In [ ]:
import numpy as np
import random
from itertools import permutations
import statistics
import time

k=3
l=12

def genLandscape():
  landscape=[random.randrange(1,100) for i in range(2000)];
  return landscape

def genAllAgents():
  nums=range(1,l+1);  
  agents=list(permutations(nums,k));
  print("Generated",len(agents)," agents");
  return agents

def playSingle(landscape,agent,x):
  fail=0
  ind=0
  curScore=landscape[x]
  l_s=len(landscape)
  l_a=len(agent)
  while fail<l_a:

    num=agent[ind]
    nextX=(x+num)%l_s;    
    nextScore=landscape[nextX];    

    if (nextScore>curScore):
      x=nextX
      curScore=nextScore    
      fail=0
    else:
      fail=fail+1        

    ind=(ind+1)%l_a

  return curScore,x

def playGroup(landscape,agents,x):
  fail=0
  ind=0
  curScore=landscape[x]
  l_as=len(agents)
  while fail<l_as:  
    nextScore,nextX=playSingle(landscape,agents[ind],x);

    if (nextX!=x):
      x=nextX
      curScore=nextScore
      fail=0      
    else:
      fail=fail+1;          

    ind=(ind+1)%l_as

  return curScore,x

def calcAgentScore(landscape,agent): 
  scores=[playSingle(landscape,agent,x)[0] for x in range(len(landscape))];  
  return statistics.mean(scores);

def calcGroupScore(landscape,agents):  
  scores=[playGroup(landscape,agents,x)[0] for x in range(len(landscape))];
  return statistics.mean(scores);  

def scoreAndSortAgents(landscape,agents):
  scores=[calcAgentScore(landscape,agents[i]) for i in range(len(agents))];
  agentScores=list(zip(scores,agents));
  agentScores.sort(reverse=True);
  return agentScores;


def printStats(name,scores):  
  print(name,"mean:",statistics.mean(scores),"median:",statistics.median(scores));


def runSim(repeatCount,printCount):
  
  expertScores=[];
  randScores=[];

  agents=genAllAgents();

  groupSize=9

  randWins=0

  start=time.clock();

  for rep in range(repeatCount):
    
    ls=genLandscape();


    agentScores=scoreAndSortAgents(ls,agents);
    topAgents=[x for _,x in agentScores[0:groupSize]]

    s1=calcGroupScore(ls,topAgents)    
  
    randAgents=random.choices(agents,k=groupSize);
    
    s2=calcGroupScore(ls,randAgents)    
  
    randWins+=s2>s1

    expertScores.append(s1);
    randScores.append(s2);

    if ((rep+1)%printCount==0):
      end=time.clock();
      print("Finished iter",rep+1,"avg. iter time:",(end-start)/(rep+1))
      
      printStats("Experts",expertScores);
      printStats("Rand groups",randScores);      
      print("Random win rate",randWins/(rep+1));


# Simulate 100 landscapes, print stats after every 10
%prun runSim(100,10);




Generated 1320  agents
Finished iter 10 avg. iter time: 6.677725700000019
Experts mean: 91.57785 median: 91.84549999999999
Rand groups mean: 93.8508 median: 93.7315
Random win rate 1.0
Finished iter 20 avg. iter time: 6.681056399999989
Experts mean: 92.053825 median: 92.58449999999999
Rand groups mean: 93.8646 median: 93.87375
Random win rate 0.95
Finished iter 30 avg. iter time: 6.686187566666664
Experts mean: 91.98746666666666 median: 92.28825
Rand groups mean: 93.73683333333334 median: 93.711
Random win rate 0.9666666666666667
Finished iter 40 avg. iter time: 6.687133700000004
Experts mean: 91.74182499999999 median: 91.97025
Rand groups mean: 93.8274625 median: 93.91175
Random win rate 0.975
Finished iter 50 avg. iter time: 6.687710420000003
Experts mean: 91.79311 median: 92.054
Rand groups mean: 93.86373 median: 93.9575
Random win rate 0.96
Finished iter 60 avg. iter time: 6.6863674999999985
Experts mean: 91.83596666666666 median: 92.0955
Rand groups mean: 93.883575 median: 93.982
